# Week 3 Mini Notebook: Searching Bigrams

## Download some Jane Austen Novels

In [2]:
import nltk, numpy, re, matplotlib# , num2words

In [3]:
cd /scratch/group/history/hist_3368-jguldi

[Errno 13] Permission denied: '/scratch/group/history/hist_3368-jguldi'
/users/dbalut/digital-history/hist3368-week3-controlled-vocab


In [4]:
#download some data

with open('senseandsensibility.txt', 'r') as myfile:
    sas_data = myfile.read().split('\n\n"I suppose you know, ma\'am, that Mr. Ferrars is married"\n\nIt _was_ Edward\n\n"Everything in such respectable condition"\n\n ')[1].split('THE END')[0].strip()

with open('emma.txt', 'r') as myfile:
    emma_data = myfile.read().split('CHAPTER I')[1].split('FINIS')[0].strip()

with open('prideandprejudice.txt', 'r') as myfile:
    pap_data = myfile.read().split('CHAPTER I')[1].split('End of the Project Gutenberg EBook of Pride and Prejudice, by Jane Austen')[0].strip()


FileNotFoundError: [Errno 2] No such file or directory: 'senseandsensibility.txt'

Make sure that your data matches what you think it should.

In [5]:
# printing only first 2000 characters.
sas_data[:2000]

NameError: name 'sas_data' is not defined

Looks good!

Isn't it getting tired, retyping the same command for each novel? Let's throw them all into one data set so we can loop through them.


In [6]:
data = [sas_data, emma_data, pap_data]
data[0][:2000]

NameError: name 'sas_data' is not defined

There still appear to be some errors where spaces have been replaced by "\n".  We'll get rid of those in a second.

In [112]:
for i in range(len(data)):
    data[i] = data[i].replace('\n', ' ')
data[0][:2000]

"*       *       *       *       *     CHAPTER I   The family of Dashwood had long been settled in Sussex. Their estate was large, and their residence was at Norland Park, in the centre of their property, where, for many generations, they had lived in so respectable a manner as to engage the general good opinion of their surrounding acquaintance. The late owner of this estate was a single man, who lived to a very advanced age, and who for many years of his life, had a constant companion and housekeeper in his sister. But her death, which happened ten years before his own, produced a great alteration in his home; for to supply her loss, he invited and received into his house the family of his nephew Mr. Henry Dashwood, the legal inheritor of the Norland estate, and the person to whom he intended to bequeath it. In the society of his nephew and niece, and their children, the old Gentleman's days were comfortably spent. His attachment to them all increased. The constant attention of Mr. a

Next, let's split the text into words and print the first word of each.

In [113]:
import pandas

for novel in data:
    words = novel.split()
    print(words[:20]) 


['*', '*', '*', '*', '*', 'CHAPTER', 'I', 'The', 'family', 'of', 'Dashwood', 'had', 'long', 'been', 'settled', 'in', 'Sussex.', 'Their', 'estate', 'was']
['Emma', 'Woodhouse,', 'handsome,', 'clever,', 'and', 'rich,', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition,', 'seemed', 'to', 'unite', 'some', 'of', 'the', 'best']
['.', 'It', 'is', 'a', 'truth', 'universally', 'acknowledged,', 'that', 'a', 'single', 'man', 'in', 'possession', 'of', 'a', 'good', 'fortune,', 'must', 'be', 'in']


## Cleaning the Novels

Now, let's lowercase the text and get rid of punctuation

In [114]:
import re

for i in range(len(data)):
    # data[i] is the current novel
    data[i] = data[i].lower() # force to lowercase
    data[i] = re.sub('[\",.;:?([)\]_*]', '', data[i]) # remove punctuation and special characters with regular expression


### Stemming

Stemming is the process of removing suffices, like "ed" or "ing".

We will use another standard NLTK package, PorterStemmer, to do the stemming.



In [115]:
from nltk.stem import PorterStemmer

st = PorterStemmer()

stemmed_list = []

for novel in data:
    words = novel.split()
    for word in words:
        stemmed = st.stem(word)
        stemmed_list.append(stemmed)
        
stemmed_list[:20] # i have changed this so you print just the first words
# printing all the words is actually way more computer intensive than it may seem

['chapter',
 'i',
 'the',
 'famili',
 'of',
 'dashwood',
 'had',
 'long',
 'been',
 'settl',
 'in',
 'sussex',
 'their',
 'estat',
 'wa',
 'larg',
 'and',
 'their',
 'resid',
 'wa']

As we can see, "universal" becomes "univers" (which means that "universally" will be counted with "universal" and "universe") and "single" becomes "singl" (which means it would be counted with "singled").  But "acknowledged" has been left as it is."

Let's see what the word counts look like now.

In [1]:
from collections import Counter

for novel in data:
    words = novel.split()
    count = Counter(words)
    print(count.most_common(10))

NameError: name 'data' is not defined

# Counting Words and N-Grams

Let's see what the word counts look like now.

In [119]:
from collections import Counter

for novel in data:
    words = novel.split()
    count = Counter(words)
    print(count.most_common(10))

[('the', 4092), ('to', 4090), ('of', 3573), ('and', 3419), ('her', 2522), ('a', 2048), ('i', 1948), ('in', 1937), ('was', 1848), ('it', 1701)]
[('and', 107), ('to', 102), ('a', 92), ('of', 90), ('the', 81), ('her', 61), ('i', 49), ('you', 48), ('it', 45), ('in', 43)]
[('you', 31), ('of', 29), ('to', 22), ('a', 21), ('the', 18), ('and', 17), ('i', 17), ('that', 15), ('it', 14), ('is', 14)]


### N-Grams

Sometimes we want to look for multi-word phrases instead of individual words.  For example, if we're researching the living spaces of Jane Austen's England, we definitely want to know whether she refers to "dining rooms" or "bed-rooms" (which our punctuation clean-up might have turned into separate words, depending on what we did).

In [120]:
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

lemmatizer = WordNetLemmatizer()
three_grams_list = []

for novel in data:
    # Get the first 20 words of the novel.
    words = novel.split(maxsplit=20)
    
    # Delete the last entry of the list as it contains the rest of the novel's text.
    del words[-1]
    
    # Lemmatize
    lemmatized_words = []
    for word in words:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    
    # Join the lemmatized words back into text.
    text = ' '.join(lemmatized_words)
    
    # Collect the n-grams and extend it to our list of n grams
    three_grams = TextBlob(text).ngrams(n=3)
    three_grams_list.extend(three_grams)

three_grams_list[:20]

[WordList(['chapter', 'i', 'the']),
 WordList(['i', 'the', 'family']),
 WordList(['the', 'family', 'of']),
 WordList(['family', 'of', 'dashwood']),
 WordList(['of', 'dashwood', 'had']),
 WordList(['dashwood', 'had', 'long']),
 WordList(['had', 'long', 'been']),
 WordList(['long', 'been', 'settled']),
 WordList(['been', 'settled', 'in']),
 WordList(['settled', 'in', 'sussex']),
 WordList(['in', 'sussex', 'their']),
 WordList(['sussex', 'their', 'estate']),
 WordList(['their', 'estate', 'wa']),
 WordList(['estate', 'wa', 'large']),
 WordList(['wa', 'large', 'and']),
 WordList(['large', 'and', 'their']),
 WordList(['and', 'their', 'residence']),
 WordList(['their', 'residence', 'wa']),
 WordList(['emma', 'woodhouse', 'handsome']),
 WordList(['woodhouse', 'handsome', 'clever'])]

In [121]:
for novel in data:
    bigrams = TextBlob(novel).ngrams(n=2)
    bigram_counter = Counter()
    for bigram in bigrams:
        # Join the bigram into a string as it is a WordList object.
        bigram_text = ' '.join(bigram)
        # Update the count.
        bigram_counter.update([bigram_text])

    print(bigram_counter.most_common(10))


[('to be', 436), ('of the', 431), ('in the', 359), ('it was', 281), ('of her', 277), ('to the', 242), ('mrs jennings', 237), ('to her', 231), ('i am', 224), ('she was', 209)]
[('” “', 29), ('miss taylor', 23), ('mr knightley', 13), ('of her', 12), ('mr weston', 12), ('of the', 10), ('to have', 9), ('it was', 9), ('her father', 9), ('she had', 9)]
[('my dear', 8), ('that he', 6), ('mr bennet', 6), ('you must', 5), ('of them', 5), ('it is', 4), ('do not', 4), ('how can', 4), ('will be', 4), ('of the', 3)]


Notice that the for loop outputs three lists of ten -- the top ten bigrams for each novel.  The output is a 'dictionary' type.  What if you wanted it as a simple list of bigrams?

In [122]:
bigrams_as_text = []

for novel in data:
    bigrams = TextBlob(novel).ngrams(n=2)
    for bigram in bigrams:
        bigram_text = ' '.join(bigram)
        bigrams_as_text.append(bigram_text)

bigrams_as_text[:20]

['chapter i',
 'i the',
 'the family',
 'family of',
 'of dashwood',
 'dashwood had',
 'had long',
 'long been',
 'been settled',
 'settled in',
 'in sussex',
 'sussex their',
 'their estate',
 'estate was',
 'was large',
 'large and',
 'and their',
 'their residence',
 'residence was',
 'was at']

Now that we have all the bigrams in one list, we can also count the overall top bigrams

In [123]:
bigram_count = Counter(bigrams_as_text)
top_twenty_bigrams = bigram_count.most_common(20)
top_twenty_bigrams

[('to be', 448),
 ('of the', 444),
 ('in the', 369),
 ('of her', 291),
 ('it was', 290),
 ('to the', 244),
 ('mrs jennings', 237),
 ('to her', 234),
 ('i am', 234),
 ('she was', 213),
 ('of his', 209),
 ('i have', 203),
 ('it is', 194),
 ('she had', 193),
 ('could not', 167),
 ('on the', 161),
 ('have been', 161),
 ('in a', 161),
 ('and the', 160),
 ('at the', 160)]

What if we only want the bigrams that include the word "she"?

In [124]:
she_bigrams = []

for bigram in bigrams_as_text:
    if "she" in bigram: # notice the space after she.  It
        she_bigrams.append(bigram)
        
Counter(she_bigrams).most_common(5)

[('she was', 213),
 ('she had', 193),
 ('that she', 122),
 ('as she', 116),
 ('she could', 108)]

Unfortunately, the above code won't work for 'he,' because it will pick up other words that contain 'he.'

In [125]:
he_bigrams = []

for bigram in bigrams_as_text:
    if "he" in bigram: # notice the space after she.  It
        he_bigrams.append(bigram)
        
Counter(he_bigrams).most_common(5)

[('of the', 444),
 ('in the', 369),
 ('of her', 291),
 ('to the', 244),
 ('to her', 234)]

The solution is to use "regular expressions," which are ways of coding the details of language.  You can communicate to the computer about such needs as detecting the beginning or end of a word by using two backslashes (an "escape" to tell the computer not to take the next letter literally) and "b" for "boundary."  If you tell the computer to find a "boundary" in this way, it will look for both spaces and for the end of strings.

Notice how I use two "\\\b"'s below to tell the computer to look for the word "he" but not "her" or "the." Python use the 're' package to detect regular expressions, and the .compile() and .match() commands

In [126]:
import re
pattern = re.compile("\\bhe\\b") #  notice the .compile() and the "escapes"+b to signify "word boundary"

In [127]:
he_bigrams = []

for bigram in bigrams_as_text:
    if pattern.match(bigram): # notice the use of .match()
        he_bigrams.append(bigram)
        
Counter(he_bigrams).most_common(5)

[('he was', 126),
 ('he had', 113),
 ('he is', 75),
 ('he has', 49),
 ('he did', 37)]

# Exercise

Write a for loop that looks for bigrams, trigrams, up to nine-grams, and counts how many there are in Jane Austen.  What are the longest phrases that are repeated more than 3 times across her corpus? 